### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline


In [2]:
# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')


In [3]:
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [4]:
# cell for work
print((movies.shape,reviews.shape))
print(movies[["movie_id"]].nunique())

((35917, 3), (876365, 4))
movie_id    35917
dtype: int64


In [5]:
# cell for work
print(movies.head(5))
print(reviews.head(5))
print(movies[["genre"]].nunique())
print(reviews[["user_id"]].nunique())


  movie_id                                              movie  \
0  0000008      Edison Kinetoscopic Record of a Sneeze (1894)   
1  0000010                La sortie des usines Lumière (1895)   
2  0000012                      The Arrival of a Train (1896)   
3       25  The Oxford and Cambridge University Boat Race ...   
4  0000091                         Le manoir du diable (1896)   

               genre  
0  Documentary|Short  
1  Documentary|Short  
2  Documentary|Short  
3                NaN  
4       Short|Horror  
  user_id  movie_id  rating   timestamp
0       1   0114508       8  1381006850
1       2   0102926       9  1590148016
2       2   0208092       5  1586466072
3       2   0358273       9  1579057827
4       2  10039344       5  1578603053
genre    2754
dtype: int64
user_id    68310
dtype: int64


In [6]:
# more work
reviews[["rating"]].isnull().sum()

rating    0
dtype: int64

In [7]:
# and still more
reviews[["rating"]].mean(),reviews[["rating"]].max()

(rating    7.316574
 dtype: float64,
 rating    10
 dtype: int64)

In [8]:
# and even more - open additional cells as necessary

In [9]:
# Use your findings to match each variable to the correct statement in the dictionary
a = 53968
b = 10
c = 7
d = 31245
e = 15
f = 0
g = 4
h = 712337
i = 28

dict_sol1 = {
'The number of movies in the dataset': 
'The number of ratings in the dataset':
'The number of different genres':
'The number of unique users in the dataset':
'The number missing ratings in the reviews dataset':
'The average rating given across all ratings':
'The minimum rating given across all ratings':
'The maximum rating given across all ratings':
}

# Check your solution
t.q1_check(dict_sol1)

SyntaxError: invalid syntax (<ipython-input-9-2e2cfa23be12>, line 14)

#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp
* Create month/day/year columns from timestamp as dummies

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [10]:
# cell for work
print(movies[["movie"]].isnull().sum())
movies.dropna(inplace = True) 
# no null in titles!
# movies[["movie"]].iloc[0].apply(str)
new_cols=movies["movie"].astype(str).str.split("(.+)\((.+)\d\d\)", n=1,expand=True)
# import re
# test='Edison Kinetoscopic Record of a Sneeze (1894)'
# re.findall("(.+)\((.+)\)",test)
new_cols[1]

movie    0
dtype: int64


0        Edison Kinetoscopic Record of a Sneeze 
1                  La sortie des usines Lumière 
2                        The Arrival of a Train 
4                           Le manoir du diable 
5                             Une nuit terrible 
                          ...                   
35910                                   Bulbbul 
35911              A Hundred Years of Happiness 
35913              Bhanumathi &amp; Ramakrishna 
35914                                      8:46 
35915                                  Homemade 
Name: 1, Length: 35672, dtype: object

In [11]:
# movies["movie"] = movies["movie"].astype(str).str.split("\s\(\s\)", n=1)
new_cols[1],new_cols[2]
dummied_years = pd.get_dummies(new_cols[2], prefix='century', prefix_sep='_') 
dummied_years

,century_18,century_19,century_20
0,1,0,0
1,1,0,0
2,1,0,0
4,1,0,0
5,1,0,0
...,...,...,...
35910,0,0,1
35911,0,0,1
35913,0,0,1
35914,0,0,1


In [12]:
# and a few more below, which you can use as necessary
# number of different genres
genres = []
for val in movies.genre:
    try:
        genres.extend(val.split('|'))
    except AttributeError:
        pass

# we end up needing this later
genres = set(genres)
print("The number of genres is {}.".format(len(genres)))

# Function to split and return values for columns
def split_genres(val):
    try:
        if val.find(gene) >-1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0

# Apply function for each genre
for gene in genres:        
    movies[gene] = movies['genre'].apply(split_genres)

movies

The number of genres is 28.


,movie_id,movie,genre,War,Comedy,Crime,Adult,Short,Music,Action,...,Horror,Fantasy,Musical,Sci-Fi,Game-Show,Family,Western,News,Documentary,Thriller
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0000091,Le manoir du diable (1896),Short|Horror,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
5,0000131,Une nuit terrible (1896),Short|Comedy|Horror,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35910,12393526,Bulbbul (2020),Horror,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
35911,12397778,A Hundred Years of Happiness (2020),Documentary,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
35913,12498582,Bhanumathi &amp; Ramakrishna (2020),Comedy|Drama|Romance,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35914,12498618,8:46 (2020),Comedy,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
movies["movie"]=new_cols[1]
# movies.drop(columns=["genre"],inplace=True)
print((movies.shape,dummied_years.shape))
movies=pd.concat([movies,dummied_years],axis=1)

((35672, 31), (35672, 3))


,movie_id,movie,genre,War,Comedy,Crime,Adult,Short,Music,Action,...,Sci-Fi,Game-Show,Family,Western,News,Documentary,Thriller,century_18,century_19,century_20
0,0000008,Edison Kinetoscopic Record of a Sneeze,Documentary|Short,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0
1,0000010,La sortie des usines Lumière,Documentary|Short,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0
2,0000012,The Arrival of a Train,Documentary|Short,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0
4,0000091,Le manoir du diable,Short|Horror,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
5,0000131,Une nuit terrible,Short|Comedy|Horror,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35910,12393526,Bulbbul,Horror,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
35911,12397778,A Hundred Years of Happiness,Documentary,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
35913,12498582,Bhanumathi &amp; Ramakrishna,Comedy|Drama|Romance,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
35914,12498618,8:46,Comedy,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [14]:
movies=pd.concat([movies,dummied_years],axis=1)

In [15]:
import datetime

change_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')

reviews['date'] = reviews['timestamp'].apply(change_timestamp)

In [16]:
# now reviews and movies are the final dataframes with the necessary columns
reviews.to_csv('./reviews_clean.csv')
movies.to_csv('./movies_clean.csv')

In [17]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date
0           0        1    114508       8  1381006850  2013-10-05 17:00:50
1           1        2    102926       9  1590148016  2020-05-22 07:46:56
2           2        2    208092       5  1586466072  2020-04-09 17:01:12
3           3        2    358273       9  1579057827  2020-01-14 22:10:27
4           4        2  10039344       5  1578603053  2020-01-09 15:50:53
   Unnamed: 0  movie_id                                    movie  \
0           0         8  Edison Kinetoscopic Record of a Sneeze    
1           1        10            La sortie des usines Lumière    
2           2        12                  The Arrival of a Train    
3           4        91                     Le manoir du diable    
4           5       131                       Une nuit terrible    

                 genre  War  Comedy  Crime  Adult  Short  Music  ...  Sci-Fi  \
0    Documentary|Short    0       0      0      0      1      0  ..